In [1]:
# -*- coding: utf-8 -*-
"""
This example demonstrates the use of ImageView, which is a high-level widget for 
displaying and analyzing 2D and 3D data. ImageView provides:

  1. A zoomable region (ViewBox) for displaying the image
  2. A combination histogram and gradient editor (HistogramLUTItem) for
     controlling the visual appearance of the image
  3. A timeline for selecting the currently displayed frame (for 3D data only).
  4. Tools for very basic analysis of image data (see ROI and Norm buttons)

"""
## Add path to library (just for examples; you do not need this)
#import initExample

import numpy as np
from pyqtgraph.Qt import QtCore, QtGui
import pyqtgraph as pg

# Interpret image data as row-major instead of col-major
pg.setConfigOptions(imageAxisOrder='row-major')

app = QtGui.QApplication([])

## Create window with ImageView widget
win = QtGui.QMainWindow()
win.resize(800,800)
imv = pg.ImageView()

def refresh():
    global imv
    _item = imv.getImageItem()
    _view = imv.getView()
    print(_view.itemBoundingRect(_item))
    
_roi = pg.ROI([0,0,200,200])
_roi.addScaleHandle([1,1],[0,0])
_roi.sigRegionChanged.connect(refresh)
imv.addItem(_roi)

win.setCentralWidget(imv)
win.show()
win.setWindowTitle('pyqtgraph example: ImageView')

## Create random 3D data set with noisy signals
img = pg.gaussianFilter(np.random.normal(size=(200, 200)), (5, 5)) * 20 + 100
img = img[np.newaxis,:,:]
decay = np.exp(-np.linspace(0,0.3,100))[:,np.newaxis,np.newaxis]
data = np.random.normal(size=(100, 200, 200))
data += img * decay
data += 2

## Add time-varying signal
sig = np.zeros(data.shape[0])
sig[30:] += np.exp(-np.linspace(1,10, 70))
sig[40:] += np.exp(-np.linspace(1,10, 60))
sig[70:] += np.exp(-np.linspace(1,10, 30))

sig = sig[:,np.newaxis,np.newaxis] * 3
data[:,50:60,30:40] += sig


## Display the data and assign each frame a time value from 1.0 to 3.0
imv.setImage(data, xvals=np.linspace(1., 3., data.shape[0]))

## Set a custom color map
colors = [
    (0, 0, 0),
    (45, 5, 61),
    (84, 42, 55),
    (150, 87, 60),
    (208, 171, 141),
    (255, 255, 255)
]
cmap = pg.ColorMap(pos=np.linspace(0.0, 1.0, 6), color=colors)
imv.setColorMap(cmap)

## Start Qt event loop unless running in interactive mode.
if __name__ == '__main__':
    import sys
    if (sys.flags.interactive != 1) or not hasattr(QtCore, 'PYQT_VERSION'):
        QtGui.QApplication.instance().exec_()


PyQt5.QtCore.QRectF(-8.881784197001252e-16, 3.552713678800501e-15, 200.0, 200.00000000000003)
PyQt5.QtCore.QRectF(-8.881784197001252e-16, 3.552713678800501e-15, 200.0, 200.00000000000003)
PyQt5.QtCore.QRectF(-8.881784197001252e-16, 3.552713678800501e-15, 200.0, 200.00000000000003)
PyQt5.QtCore.QRectF(-8.881784197001252e-16, 3.552713678800501e-15, 200.0, 200.00000000000003)
PyQt5.QtCore.QRectF(-8.881784197001252e-16, 3.552713678800501e-15, 200.0, 200.00000000000003)
PyQt5.QtCore.QRectF(-8.881784197001252e-16, 3.552713678800501e-15, 200.0, 200.00000000000003)
PyQt5.QtCore.QRectF(-8.881784197001252e-16, 3.552713678800501e-15, 200.0, 200.00000000000003)
PyQt5.QtCore.QRectF(-8.881784197001252e-16, 3.552713678800501e-15, 200.0, 200.00000000000003)
PyQt5.QtCore.QRectF(-8.881784197001252e-16, 3.552713678800501e-15, 200.0, 200.00000000000003)
PyQt5.QtCore.QRectF(-8.881784197001252e-16, 3.552713678800501e-15, 200.0, 200.00000000000003)
PyQt5.QtCore.QRectF(-8.881784197001252e-16, 3.55271367880050

In [9]:
_item = imv.getImageItem()
_view = imv.getView()

In [10]:
print(_view.itemBoundingRect(_item))

PyQt5.QtCore.QRectF(0.0, -7.105427357601002e-15, 200.0, 199.99999999999997)


In [6]:
view_widget = _view.getViewWidget()

In [4]:
my_view.screenGeometry()

PyQt5.QtCore.QRect(1022, 296, 679, 745)